# mymodelnet40
- Prerocessed dataset of [modelnet40_normal_resampled](https://shapenet.cs.stanford.edu/media/modelnet40_normal_resampled.zip) for time-saving training.

## Import packages.

In [10]:
import os
import h5py
import numpy as np
import k3d

import torch

# my packages
from libs import tpcpath
from torchpcp.utils import pytorch_tools
from torchpcp.datasets.PointNet2.ModelNet import ModelNet40
from torchpcp.datasets.PointNet2.ModelNet import MyModelNet40

## Set a seed.

In [3]:
pytorch_tools.set_seed(0)
device = pytorch_tools.select_device("cuda")

## Visualizetion function

In [4]:
def visualize_pc(pc, colors=None):
    if colors is None:
        colors = np.full(len(pc), 0x000000, dtype=np.float64) # white: 0xFFFFFF
    plot = k3d.plot()
    points = k3d.points(pc, colors.astype(np.float32), point_size=0.04, shader='flat')
    plot += points
    plot.display()


## Load Dataset of modelnet40_normal_resampled.

In [ ]:
path = "/mnt/databox/datasets/ModelNet/modelnet40_normal_resampled"
# path = "/path/to/modelnet40_normal_resampled"
datasets = ModelNet40(path)

## Define output path.

In [15]:
# output_path = "/path/to/outputs/"
output_path = "/mnt/databox/datasets/ModelNet/"
preprocessed_dataset = "mymodelnet40"
preprocessed_dataset_path = os.path.join(output_path, preprocessed_dataset)


## Preprocess the dataset.

In [6]:
def create_modelnet_dataset(file_name, dataset):
    loader = range(len(dataset))
    # loader = tqdm(loader, desc=file_name)
    with h5py.File(file_name, "w") as f:
        point_clouds = []
        labels = []
        for i in loader:
            points, label = dataset[i]
            point_clouds.append(points)
            labels.append(label)

        f.create_dataset("point_clouds", data=point_clouds, compression='gzip', compression_opts=1, dtype="float32")
        f.create_dataset("labels", data=labels, compression='gzip', compression_opts=1, dtype="uint8")


In [12]:
os.mkdir(preprocessed_dataset_path)
train_dataset_path = os.path.join(preprocessed_dataset_path, "train_modelnet40.h5")
create_modelnet_dataset(train_dataset_path, datasets["train"])
test_dataset_path = os.path.join(preprocessed_dataset_path, "test_modelnet40.h5")
create_modelnet_dataset(test_dataset_path, datasets["test"])


## Use MyModelNet40 (preprocessed modelnet40_normal_resampled).

In [18]:
my_dataset = MyModelNet40(preprocessed_dataset_path)

## Visualize a point cloud.

In [34]:
points, label = my_dataset["train"][3001]

In [35]:
visualize_pc(points[:, :3])

Output()

In [ ]:
visualize_pc(points[:, 3:6])